<a href="https://colab.research.google.com/github/paro2708/SER517_Group35_Capstone/blob/GazeRefineNet/EyeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To add

# EyeNet

EyeNet model to be used to tranfer learning to GazeRefineNet model

In [2]:
import torchvision.models as models
import torch
from torch import nn
import numpy as np
import math

In [15]:
class EyeNet(nn.Module):
  def __init__(self, use_rnn=True):
    super(EyeNet, self).__init__()
    self.resnet = models.resnet18(
        pretrained=False,        # Not using pre-trained weights
        num_classes=128,        # (IMP)Number of output classes - needs to be defined based on number of eye features needed
        norm_layer=nn.InstanceNorm2d,  # Normalization layer - layers need to be added
    )
    self.use_rnn = use_rnn

    # Optional recurrent component - GRUCell
    if use_rnn:
            self.rnn = nn.GRUCell(input_size=128, hidden_size=128)
            self.fc_gaze = nn.Linear(128, 3)  # Output size for gaze direction
    else:
      self.fc_gaze = nn.Sequential(
            nn.Linear(128, 128),
            nn.SELU(inplace=True),
            nn.Linear(128, 2, bias=False),
            nn.Tanh(),
        )  # Output size for gaze direction
    self.fc_pupil = nn.Sequential(
            nn.Linear(128, 128),
            nn.SELU(inplace=True),
            nn.Linear(128, 1),
            nn.ReLU(inplace=True),
        )  # Output size for pupil size

  def forward(self, input_eye_image, rnn_output=None):

    features = self.resnet(input_eye_image)
    if self.use_rnn:
      batch_size, feature_size = features.shape
      hidden = torch.zeros(batch_size, 128)
      previous_results = []
      output=[]
      rnn_features=features
      for i in range(batch_size):  # Loop through layers
        output.append(self.rnn(rnn_features, hidden))  # Forward pass through GRU cell
        rnn_features=output
      features=rnn_features


    gaze_direction = self.fc_gaze(features)
    pupil_size = self.fc_pupil(features)
    return gaze_direction, pupil_size

   # Define custom loss functions
  def gaze_loss(g, g_hat):
    loss = torch.mean(torch.acos(torch.sum(g * g_hat, dim=1) / (torch.norm(g, dim=1) * torch.norm(g_hat, dim=1))) * (180 / torch.pi))
    return loss

  def pupil_loss(p, p_hat):
    loss = torch.mean(torch.abs(p - p_hat))
    return loss

#To test if the models can run image data for eye coordinates
#eyenet = EyeNet(use_rnn=True)  # You can set use_rnn to True or False based on your requirement

# Generate sample input data (e.g., random tensor with appropriate dimensions)
# assuming batch size = 1, 3 channels (RGB), and image size 224x224
# input_data = torch.randn(1, 3, 224, 224)

# # Pass the input data through the EyeNet model to get predictions
# gaze_direction, pupil_size = eyenet(input_data)

# magnitude = torch.norm(gaze_direction, p=2)

# # Print the predictions
# print("Predicted Gaze Direction:", gaze_direction)
# print("Predicted Pupil Size:", pupil_size)
# print("Normalized Gaze Direction Magnitude(in radians):", magnitude.item())




In [16]:
from torchvision import transforms
from PIL import Image

eyenet= EyeNet()

# Load and preprocess the image
image_path = '/WhatsApp Image 2024-03-11 at 13.11.44_0b8699f8.jpg'
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = Image.open(image_path)
img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension

# Assuming the EyeNet model is already defined and initialized as eyenet
gaze_direction, pupil_size = eyenet(img_tensor.squeeze(0))

print("Predicted Gaze Direction:", gaze_direction)
print("Predicted Pupil Size:", pupil_size)
magnitude = torch.norm(gaze_direction, p=2)
magnitude = magnitude *(180/np.pi)
print("Normalized Gaze Direction Magnitude(in radians):", magnitude.item())


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x1 and 512x128)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Training Script